In [1]:
import os

import gym
import numpy as np
import pandas as pd
import seaborn as sns

from ray import init, rllib, tune, shutdown

In [2]:
from attack_simulator.agents import DEFENDERS
from attack_simulator.env import AttackSimulationEnv
from attack_simulator.graph import AttackGraph, SIZES

In [3]:
class AgentPolicy(rllib.policy.Policy):
    def __init__(self, observation_space, action_space, config):
        super().__init__(observation_space, action_space, config)
        agent_config = dict(
            input_dim=observation_space.shape[0],
            num_actions=action_space.n,
            random_seed=config["seed"],
            attack_graph=config["env_config"]["attack_graph"],
        )
        self._agent = DEFENDERS[config["agent_type"]](agent_config)

    def compute_actions(self, observations, *args, **kwargs):
        # FIXME: use a `numpy` array as a temporary workaround for
        #        https://github.com/ray-project/ray/issues/10100
        return np.array([self._agent.act(obs) for obs in observations]), [], {}

    def get_weights(self):
        return {}

    def set_weights(self, weights):
        pass


def template_agent(agent_type):
    default_config = rllib.agents.trainer.with_common_config(dict(agent_type=agent_type))
    return rllib.agents.trainer_template.build_trainer(
        name=agent_type,
        default_policy=AgentPolicy,
        default_config=default_config,
    )

In [4]:
from copy import deepcopy

class AttackSimAlphaZeroEnv(AttackSimulationEnv):
    def set_state(self, state):
        (
            self.simulation_time,
            self.ttc_remaining,
            self.attack_surface,
            self.attack_state,
            self.service_state,
            self._observation,
        ) = deepcopy(state)

    def get_state(self):
        state = (
            self.simulation_time,
            self.ttc_remaining,
            self.attack_surface,
            self.attack_state,
            self.service_state,
            self._observation,
        )
        return deepcopy(state)
    

class AlphaZeroWrapper(gym.Env):
    def __init__(self, config):
        self.env = config["env_class"](config)
        self.action_space = self.env.action_space
        assert isinstance(
            self.action_space, gym.spaces.Discrete
        ), "AlphaZero requires a Discrete action space"
        shape = (self.action_space.n,)
        self.observation_space = gym.spaces.Dict(
            dict(obs=self.env.observation_space, action_mask=gym.spaces.Box(0, 1, shape))
        )
        self.reward = 0
        self.mask = np.full(shape, 1, dtype="int8")

    def reset(self):
        self.reward = 0
        observation = self.env.reset()
        return dict(obs=observation, action_mask=self.mask)

    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        self.reward += reward
        reward = self.reward if done else 0
        return dict(obs=observation, action_mask=self.mask), reward, done, info

    def set_state(self, state):
        env_state, self.reward = state
        self.env.set_state(env_state)
        return dict(obs=self.env.observation, action_mask=self.mask)

    def get_state(self):
        return self.env.get_state(), self.reward

    def close(self):
        self.env.close()

    def render(self, mode=None):
        self.env.render(mode)

    def seed(self, seed=None):
        self.env.seed(seed)

In [5]:
# 'contrib/AlphaZero' does NOT appear to work without its custom dense model
from ray.rllib.contrib.alpha_zero.models.custom_torch_models import DenseModel

rllib.models.ModelCatalog.register_custom_model("alpha_zero_dense_model", DenseModel)

In [6]:
# work around: https://github.com/ray-project/ray/issues/17618

from IPython.core.interactiveshell import InteractiveShell


class keep_ipython_sane:
    def __enter__(self):
        self.instance = InteractiveShell.instance()
        
    def __exit__(self, *args, **kwargs):
        # feel free to improve with error handling, etc.
        InteractiveShell._instance = self.instance

In [7]:
if os.path.isdir("/var/run/secrets/kubernetes.io"):  # inside k8s pod
    args = dict(address="auto")
else:
    # listen on all interfaces inside a container for port-forwarding to work
    dashboard_host = "0.0.0.0" if os.path.exists("/.dockerenv") else "127.0.0.1"
    args = dict(num_cpus=4, dashboard_host=dashboard_host)

# ALTERNATIVE: use the "Ray client" to connect to a remote cluster
# Unfortunately, JupyterNotebookReporter displays an object reference
# <IPython.core.display.HTML object> instead of content...
# --- --- ---
#
# from ray.util.client import worker
#
# worker.INITIAL_TIMEOUT_SEC = worker.MAX_TIMEOUT_SEC = 1
#
# ray_client_server = 'host.docker.internal' if os.path.exists("/.dockerenv") else '127.0.0.1'
# try:
#     init(address=f'ray://{ray_client_server}:10001')
# except Connection Error:
#     pass  # TODO: try something else...

In [8]:
from ray.tune.utils.trainable import TrainableUtil
from tqdm.auto import tqdm

rename = dict(
    agent_type="Agent",
    graph_size="Graph size",
    episode_length="Episode lengths",
    episode_reward="Returns",
)

# FIXME: temporarily omit "contrib/AlphaZero" due to https://github.com/ray-project/ray/issues/13177
# agent_types = ["contrib/AlphaZero", "R2D2", "rule-based", "random"]
agent_types = ["R2D2", "rule-based", "random"]
graphs = [AttackGraph(dict(graph_size=size)) for size in SIZES]
seeds = [0, 1, 2, 3, 6, 7, 11, 28, 42, 1337]
iterations = 10
rollouts = 10

eval_config = dict(
    evaluation_interval=1,
    evaluation_num_workers=1,
    evaluation_config=dict(explore=False, replay_sequence_length=-1),
    evaluation_num_episodes=rollouts,
)

def generate(savename):
    init(**args)
    
    frames = []
    for graph in tqdm(graphs, 'graphs'):
        for agent_type in tqdm(agent_types, f'└── {graph.graph_size}'):
            agent_name = agent_type.split("/")[-1]
            for seed in tqdm(seeds, f'\u00a0\u2001\u2001\u2001└── {agent_name}@{graph.graph_size}'):
                config = dict(
                    framework="torch",
                    model=dict(use_lstm=True),
                    env=AttackSimulationEnv,
                    env_config=dict(attack_graph=graph),
                    seed=seed,
                    num_workers=0,
                    log_level='ERROR',
                )
                if agent_type in DEFENDERS:
                    config.update(eval_config)
                    agent = template_agent(agent_type)(config=config)
                else:
                    if agent_type == "contrib/AlphaZero":
                        config["env_config"].update(env_class=AttackSimAlphaZeroEnv)
                        config.update(
                            env=AlphaZeroWrapper,
                            model=dict(custom_model="alpha_zero_dense_model"),
                            rollout_fragment_length=32,
                            train_batch_size=1280,
                            sgd_minibatch_size=64,
                        )

                    name = f'{agent_name}_{graph.graph_size}_{seed}'
                    if not os.path.exists(name):
                        config.update(num_workers=4, batch_mode="complete_episodes")
                        with keep_ipython_sane():
                            agent = rllib.agents.registry.get_trainer_class(agent_type)(config=config)
                        for _ in tqdm(range(iterations), f'\u00a0\u2001\u2001\u2001\u2001\u2001\u2001└── {name}'):
                            results = agent.train()
                            # TODO: break based on results?
                        agent.save(name)
                        del agent

                    config.update(eval_config, num_workers=0)
                    with keep_ipython_sane():
                        agent = rllib.agents.registry.get_trainer_class(agent_type)(config=config)
                        checkpoint_path = TrainableUtil.get_checkpoints_paths(name).chkpt_path[0]
                        agent.restore(checkpoint_path)

                stats = agent.evaluate()['evaluation']['hist_stats']
                frame = pd.DataFrame(dict(agent_type=agent_type, graph_size=graph.num_attacks, **stats))
                frames.append(frame)
    shutdown()
    results_df = pd.concat(frames, ignore_index=True).rename(columns=rename)
    results_df.to_csv(savename)
    return results_df

AttackGraph(en2720.yaml[tiny], 2 services, 7 attack steps)
AttackGraph(en2720.yaml[small], 5 services, 27 attack steps)
AttackGraph(en2720.yaml[medium-small], 9 services, 44 attack steps)
AttackGraph(en2720.yaml[medium], 13 services, 56 attack steps)
AttackGraph(en2720.yaml[large], 16 services, 62 attack steps)
AttackGraph(en2720.yaml[extra-large], 16 services, 65 attack steps)
AttackGraph(en2720.yaml[full], 18 services, 78 attack steps)


In [9]:
savename = "returns-agent-eval.csv"

df = generate(savename) if not os.path.exists(savename) else pd.read_csv(savename, index_col=0)

2021-09-22 13:30:33,891	INFO services.py:1263 -- View the Ray dashboard at http://172.17.0.3:8265


graphs:   0%|          | 0/7 [00:00<?, ?it/s]

└── tiny:   0%|          | 0/3 [00:00<?, ?it/s]

    └── R2D2@tiny:   0%|          | 0/10 [00:00<?, ?it/s]

2021-09-22 13:30:34,931	INFO trainer.py:726 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


...

       └── R2D2_tiny_1337:   0%|          | 0/10 [00:00<?, ?it/s]

2021-09-22 13:42:47,467	INFO trainable.py:382 -- Restored on 172.17.0.3 from checkpoint: R2D2_tiny_1337/checkpoint_000010/checkpoint-10
2021-09-22 13:42:47,470	INFO trainable.py:390 -- Current state after restoring: {'_iteration': 10, '_timesteps_total': None, '_time_total': 80.3370463848114, '_episodes_total': 733}


    └── rule-based@tiny:   0%|          | 0/10 [00:00<?, ?it/s]

       └── R2D2_small_0:   0%|          | 0/10 [00:00<?, ?it/s]


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
df

In [ ]:
sns.set(style="darkgrid", rc={"figure.figsize": (12, 8)})

In [ ]:
g = sns.lineplot(data=df, x="Graph size", y="Returns", hue="Agent", ci="sd")
g.legend(title="Agent", loc="lower left")
g.set_title("Returns vs Size (random attacker)")

In [ ]:
g = sns.lineplot(data=df, x="Graph size", y="Episode lengths", hue="Agent", ci="sd")
g.legend(title="Agent", loc="upper left")
g.set_title("Episode lengths vs Size (random attacker)")

In [ ]:
pd.set_option("display.max_columns", 32)
df.groupby("Agent").describe()